# Data collection

This notebook walks through the process of collecting the data needed for this analysis.

-------------------------

### Setup

This Jupyter Notebook is created using [Python version 3.7](https://www.python.org/downloads/release/python-370/).

First, I will import the necessary libraries to run the code. The following libraries are used:  
* [json 2.0.9](https://docs.python.org/3/library/json.html)  
* [requests 2.22.0](https://pypi.org/project/requests/) 
* [time](https://docs.python.org/3/library/time.html)  
* [pandas 0.25.3](https://pandas.pydata.org/)
* [datetime](https://docs.python.org/3/library/datetime.html)

In [1]:
import json
import requests
import time
import pandas as pd
from datetime import datetime as dt

------------

### Function definitions

This section defines functions to be used later in the file.

The `make_and_save_api_call` function makes an API call to the given url endpoint with the given parameters. The json respoonse from the call is saved with a `.json` extension to the given location and with the given file name.

In [2]:
def make_and_save_api_call(endpoint, params, filename):
    """
    inputs:
        - endoint: URL for the API call to go to
        - params: Parameters to be passed with the API call
        - filename: File path and name to save json response to
    outputs:
        - No returns. Response is saved to .json file.
    """
    call = requests.get(endpoint.format(**params))
    response = call.json()
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(response, file)

The `get_noaa_data` function makes an API call to the given url endpoint to gether the given datatype with the given header information. The json response from the API call is returned from the function call.

In [3]:
def get_noaa_data(endpoint, data_type, header):
    """
    inputs:
        - endpoint: URL for the API call to go to
        - data_type: Type of NOAA data to query for
        - header: Header information for call (Should be a dictionary with API access token)
    outputs:
        - Returns json response from call
    """
    r = requests.get(endpoint, data_type, headers=header)
    return r.text

-----------------

### College football data API Calls

Details about the college football dataset are found [here](https://github.com/BlueSCar/cfb-database) and is aggregated by GitHub user [Bill Radjewski](https://github.com/BlueSCar). This data is available under an [MIT License](https://github.com/BlueSCar/cfb-database/blob/master/LICENSE). 

The schema for the data is as follows:  


![alt text](https://github.com/BlueSCar/cfb-database/blob/master/SchemaDiagram.png?raw=true)

I accessed this data using the [Swagger API endpoints](https://api.collegefootballdata.com/api/docs/?url=/api-docs.json).

-----------------

### Game data

The first endpoint I called is the games data point accessed via `https://api.collegefootballdata.com/games`.

The games data call takes the following parameters:  

| name     | required | description |
| ---------|----------|-------------|
| year     | yes      | year/season filter for games |
| week     | no       | week filter |
| seasonType| no | season type filter|
| team     | no | team name|
| home     | no | home team filter |
| away     | no | away team filter |
| conference| no | conference abbreviation filter|
| id| no | id filter for querying a single game|

I wanted all game data so I only passed the year parameter. This returns all reqular season games from that year in the following format:  

```json
  {
    "id": 401013357,
    "season": 2018,
    "week": 1,
    "season_type": "regular",
    "start_date": "2018-08-25T21:30:00.000Z",
    "neutral_site": false,
    "conference_game": false,
    "attendance": 8684,
    "venue_id": 3985,
    "venue": "Warren McGuirk Alumni Stadium",
    "home_team": "UMass",
    "home_conference": "FBS Independents",
    "home_points": 63,
    "home_line_scores": [
      21,
      14,
      21,
      7
    ],
    "home_post_win_prob": "0.9984542422162311",
    "away_team": "Duquesne",
    "away_conference": null,
    "away_points": 15,
    "away_line_scores": [
      3,
      6,
      0,
      6
    ],
    "away_post_win_prob": "0.001545757783768864"
  }
```

I wil then make the API calls to the following endpoint:

In [4]:
games_endpoint = "https://api.collegefootballdata.com/games?year={year}"

There is no attendance data recorded for seasons before 2011. Therefore, I will be looking at the 2012-2018 seasons so I will make 7 calls, 1 each to get data for all of these years and save them to their corresponding `.json` extension files using the `make_and_save_api_call` defined above:

In [5]:
make_and_save_api_call(games_endpoint, {"year": "2012"}, "raw_data/games_endpoint_2012.json")
make_and_save_api_call(games_endpoint, {"year": "2013"}, "raw_data/games_endpoint_2013.json")
make_and_save_api_call(games_endpoint, {"year": "2014"}, "raw_data/games_endpoint_2014.json")
make_and_save_api_call(games_endpoint, {"year": "2015"}, "raw_data/games_endpoint_2015.json")
make_and_save_api_call(games_endpoint, {"year": "2016"}, "raw_data/games_endpoint_2016.json")
make_and_save_api_call(games_endpoint, {"year": "2017"}, "raw_data/games_endpoint_2017.json")
make_and_save_api_call(games_endpoint, {"year": "2018"}, "raw_data/games_endpoint_2018.json")

I will then read all of these in from their corresponding json files into DataFrames using [panda's `read_json` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html):

In [6]:
games_2012 = pd.read_json("raw_data/games_endpoint_2012.json")
games_2013 = pd.read_json("raw_data/games_endpoint_2013.json")
games_2014 = pd.read_json("raw_data/games_endpoint_2014.json")
games_2015 = pd.read_json("raw_data/games_endpoint_2015.json")
games_2016 = pd.read_json("raw_data/games_endpoint_2016.json")
games_2017 = pd.read_json("raw_data/games_endpoint_2017.json")
games_2018 = pd.read_json("raw_data/games_endpoint_2018.json")

I will then join all 7 seasons into one DataFrame named `all_games` using [panda's `concat` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) and we can preview the data available:

In [7]:
all_games = pd.concat([games_2012, games_2013, games_2014, games_2015, games_2016, games_2017, games_2018], ignore_index=True)
all_games.head()

,attendance,away_conference,away_line_scores,away_points,away_post_win_prob,away_team,conference_game,home_conference,home_line_scores,home_points,home_post_win_prob,home_team,id,neutral_site,season,season_type,start_date,venue,venue_id,week
0,12616.0,Conference USA,"[7, 28, 14, 7]",56,0.796214,UCF,False,Mid-American,"[0, 0, 7, 7]",14,0.203786,Akron,322432006,False,2012,regular,2012-08-30T19:00:00.000Z,Summa Field at InfoCision Stadium,3768.0,1
1,38393.0,SEC,"[7, 3, 0, 7]",17,0.518995,South Carolina,True,SEC,"[0, 10, 3, 0]",13,0.481005,Vanderbilt,322430238,False,2012,regular,2012-08-30T19:00:00.000Z,Vanderbilt Stadium,3973.0,1
2,12725.0,Mid-American,"[0, 13, 0, 13]",26,0.020080,Eastern Michigan,True,Mid-American,"[10, 3, 21, 3]",37,0.979920,Ball State,322432050,False,2012,regular,2012-08-30T19:00:00.000Z,Scheumann Stadium,3919.0,1
3,15250.0,None,"[10, 14, 3, 0]",27,0.369692,Southeast Missouri State,False,Mid-American,"[10, 14, 7, 7]",38,0.630308,Central Michigan,322432117,False,2012,regular,2012-08-30T19:00:00.000Z,Kelly/Shorts Stadium,3786.0,1
4,15121.0,None,"[0, 7, 0, 14]",21,0.644572,Towson,False,Mid-American,"[17, 10, 7, 7]",41,0.355428,Kent State,322432309,False,2012,regular,2012-08-30T19:00:00.000Z,Dix Stadium,3696.0,1


Since I am doing an analysis on attendance, I want to only use data with correctly recorded attendance values (should be over 0). I will check how many rows of data have attendances = 0:

In [8]:
print("There are %d rows of data with attendance = 0" % len(all_games[all_games["attendance"] == 0]))

There are 911 rows of data with attendance = 0


Since this is only a small fraction of the overall data, I will just exclude these rows from the dataset by filtering only for rows with attendance > 0:

In [9]:
all_games = all_games[all_games["attendance"] > 0]

### Venue data

The other endpoint I called is the vanues data point accessed via `https://api.collegefootballdata.com/venues`.

The venues endpoint does not take any parameters.

The venues endpoint returns a list of all stadiums that hosted games included in the dataset. An example return is the following:

```json
  {
    "id": 3601,
    "name": "Aggie Memorial Stadium",
    "capacity": 30343,
    "grass": false,
    "city": "Las Cruces",
    "state": "NM",
    "zip": "88003",
    "country_code": "US",
    "location": {
      "x": 32.2796202,
      "y": -106.7411148
    },
    "elevation": "1208.201294",
    "year_constructed": 1978,
    "dome": false
  }
```

I will then make the API call to this endpoint to gather venue level information using the same `make_and_save_api_call` function from above. I will also save this to a `.json` file and then read it in as a DataFrame to print a preview:

In [10]:
venues_endpoint = "https://api.collegefootballdata.com/venues"
make_and_save_api_call(venues_endpoint, {}, "raw_data/venues_endpoint.json")
venues = pd.read_json("raw_data/venues_endpoint.json")
venues.head()

,capacity,city,country_code,dome,elevation,grass,id,location,name,state,year_constructed,zip
0,10000,Tuskegee,US,0.0,122.800000,NaN,4747,None,Abbott Memorial Alumni Stadium,AL,1925.0,NaN
1,28500,Baton Rouge,US,0.0,20.242132,NaN,3599,"{'x': 30.5221461, 'y': -91.1896087}",Ace W. Mumford Stadium,LA,1928.0,NaN
2,30343,Las Cruces,US,0.0,1208.201294,0.0,3601,"{'x': 32.2796202, 'y': -106.7411148}",Aggie Memorial Stadium,NM,1978.0,88003.0
3,21500,Greensboro,US,0.0,235.122452,NaN,3603,"{'x': 36.0814337, 'y': -79.7700391}",Aggie Stadium,NC,1981.0,NaN
4,10849,Davis,US,0.0,11.382592,NaN,3602,"{'x': 38.5365266, 'y': -121.7627936}",Aggie Stadium,CA,2007.0,NaN


-------------------

### Game and venue data combination

All venues have a unique ID and `venue_id` is included in the games datset as well. Therefore, we can join the two datasets based on this field. 

First, we rename the `id` field in the venues dataset to `venue_id` so the name matches between the two DataFrames:

In [11]:
venues = venues.rename(columns={"id": "venue_id"})

We can then use [panda's `merge` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) to combine the two datasets on the `venue_id` column:

In [12]:
attendance_data = pd.merge(all_games, venues, on="venue_id")

We can preview the dataset now and see that the columns on the right are indeed venue level:

In [13]:
attendance_data.head()

,attendance,away_conference,away_line_scores,away_points,away_post_win_prob,away_team,conference_game,home_conference,home_line_scores,home_points,...,city,country_code,dome,elevation,grass,location,name,state,year_constructed,zip
0,12616.0,Conference USA,"[7, 28, 14, 7]",56,0.796214,UCF,False,Mid-American,"[0, 0, 7, 7]",14,...,Akron,US,0.0,321.287506,0.0,"{'x': 41.0725534, 'y': -81.5083408}",Summa Field at InfoCision Stadium,OH,2009.0,44399.0
1,9933.0,None,"[3, 3, 0, 0]",6,0.000028,Morgan State,False,Mid-American,"[14, 7, 24, 21]",66,...,Akron,US,0.0,321.287506,0.0,"{'x': 41.0725534, 'y': -81.5083408}",Summa Field at InfoCision Stadium,OH,2009.0,44399.0
2,8211.0,Mid-American,"[13, 22, 10, 11]",56,0.052170,Miami (OH),True,Mid-American,"[14, 14, 7, 14]",49,...,Akron,US,0.0,321.287506,0.0,"{'x': 41.0725534, 'y': -81.5083408}",Summa Field at InfoCision Stadium,OH,2009.0,44399.0
3,10102.0,Mid-American,"[0, 0, 17, 7]",24,0.521928,Bowling Green,True,Mid-American,"[0, 10, 0, 0]",10,...,Akron,US,0.0,321.287506,0.0,"{'x': 41.0725534, 'y': -81.5083408}",Summa Field at InfoCision Stadium,OH,2009.0,44399.0
4,7074.0,Mid-American,"[10, 10, 10, 7]",37,0.699729,Northern Illinois,True,Mid-American,"[0, 7, 0, 0]",7,...,Akron,US,0.0,321.287506,0.0,"{'x': 41.0725534, 'y': -81.5083408}",Summa Field at InfoCision Stadium,OH,2009.0,44399.0


-------------------------

### NOAA weather data API Calls

In this section, I will make calls to the NOAA weather service to retrieve gameday rain and temperature data. Details on the API can be found [here](https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted).

To complete the API call, you must request a Web Services Token by signing up with your email [here](https://www.ncdc.noaa.gov/cdo-web/token). This must be passed in a dictionary as a credential for the API call to return results.

I wanted to collect temperature and rain data for all the games in the dataset. To do so, I make a call to the Daily Summary dataset (GHCND) which contains historical sumaries. More information can be found on this dataset [here](https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets/GHCND).

The API call takes several parameters:

| name    | description     | values |
|---------|-----------------|--------|
|locationid|location identifier information|ZIP:(zip code of venue)|
|startdate|start date of data collection|(month-date-year of game)|
|enddate|end date of data collection|(month-date-year of game)|

I make API calls to this endpoint for all games included in the dataset, passing in the corresponding zip code, and game date as a string for both the start and end date as we only need information for that one day. Note: There is a threshold of API calls allowed in a period of time so I added a sleep statement for 180 seconds every 1000 calls. 

I first needed to convert the columns to the correct types to make the API call. I first conver the zip to int, following [this example](https://stackoverflow.com/questions/21287624/convert-pandas-column-containing-nans-to-dtype-int) of how to deal with integer type in Python when there are NaN values and also convert the game date to the correctly formatted date type for the API call:

In [14]:
attendance_data["zip"] = attendance_data["zip"].astype('Int64')
attendance_data["start_date"] = pd.to_datetime(attendance_data["start_date"]).dt.date

The call is thenmade using the `get_noaa_data` function defined earlier and the JSON reponses are appended to a `weather_responses` list:

In [15]:
weather_responses = []

token = "put_toke_here"
creds = dict(token=token)
dtype = "dataset"

for index, row in attendance_data.iterrows():
    endpoint = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:%s&startdate=%s&enddate=%s" % (row.zip, row.start_date, row.start_date)
    weather_responses.append(get_noaa_data(endpoint, dtype, creds))
    if index % 1000 == 0:
        time.sleep(180)

ConnectionError: HTTPSConnectionPool(host='www.ncdc.noaa.gov', port=443): Max retries exceeded with url: /cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:85287&startdate=2015-09-18&enddate=2015-09-18&dataset (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000028BE69048D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

The raw JSON values need to be parsed to pull out the precipitation and temperature data from the daily summary. To do so we need to grab the values for several fields:  
* PRCP = precipitation for that day
* TMAX = maximum temperature for that day
* TMIN = minimum temperature for that day

We can then pull out the results for these values by accessing the value for the JSON field with that identifier and aggregate them in lists for `rain`, `temp_max`, and `temp_min`:

In [ ]:
rain = []
temp_max = []
temp_min = []

for response in weather_responses:
    response_json = json.loads(response)
    if response_json:
        for i in range(len(response_json["results"])):
            if response_json["results"][i]["datatype"] == "PRCP":
                rain.append(response_json["results"][i]["value"])
            elif response_json["results"][i]["datatype"] == "TMAX":
                temp_max.append(response_json["results"][i]["value"])
            elif response_json["results"][i]["datatype"] == "TMIN":
                temp_min.append(response_json["results"][i]["value"])
    else:
        rain.append(None)
        temp_max.append(None)
        temp_min.append(None)

I will then add these rows to the resulting dataframe. In the event the json calls are not successfully completed (due to an API call threshold reached or other error) I will pad the resulting arrays with `0`'s to be the correct length and allow successful addition to the dataframe. Finally, the rain and temperature data is added to the `attendance_data` dataframe.

In [ ]:
rain = rain + [None] * (attendance_data.shape[0] - len(rain))
temp_max = temp_max + [None] * (attendance_data.shape[0] - len(temp_max))
temp_min = temp_min + [None] * (attendance_data.shape[0] - len(temp_min))

attendance_data["rain"] = rain
attendance_data["max_temp"] = temp_max
attendance_data["min_temp"] = temp_min

print("The resulting dataset has a shape: " attendance_data.shape)

-----------------

### Feature engineering

Since I want to compare attendance accross multiple programs, I needed to create a metric that could be compared. The dataset includes attedance counts which can not be used across stadiums that fit different numbers of people. Therefore I created the `attendance_percentage` metric to normalize for stadium size:

In [ ]:
attendance_data["attendance_percentage"] = attendance_data["attendance"] / attendance_data["capacity"]

I want to look at team success in relation to attendance which I will do using rankings data which was also included in the college football dataset available through the rankings endpoint. 

The rankings data call takes the following parameters:  

| name     | required | description |
| ---------|----------|-------------|
| year     | yes      | year/season filter for games |
| week     | no       | week filter |
| seasonType| no | season type filter|

I wanted all game data so I only passed the year parameter. This returns all regular season games from that year in the following format:  

```json
 {
    "season": 2019,
    "seasonType": "regular",
    "week": 8,
    "polls": [
      {
        "poll": "AP Top 25",
        "ranks": [
          {
            "rank": 1,
            "school": "Alabama",
            "conference": "SEC",
            "firstPlaceVotes": 30,
            "points": 1503
          },
          {
            "rank": 2,
            "school": "LSU",
            "conference": "SEC",
            "firstPlaceVotes": 12,
            "points": 1449
          },
          .....
       }
    ]
}
```

I wil then make the API calls to the following endpoint for all of years I am looking at using the `make_and_save_api_call` function defined above:

In [ ]:
rankings_endpoint = "https://api.collegefootballdata.com/rankings?year={year}"
make_and_save_api_call(rankings_endpoint, {"year": "2012"}, "raw_data/rankings_endpoint_2012.json")
make_and_save_api_call(rankings_endpoint, {"year": "2013"}, "raw_data/rankings_endpoint_2013.json")
make_and_save_api_call(rankings_endpoint, {"year": "2014"}, "raw_data/rankings_endpoint_2014.json")
make_and_save_api_call(rankings_endpoint, {"year": "2015"}, "raw_data/rankings_endpoint_2015.json")
make_and_save_api_call(rankings_endpoint, {"year": "2016"}, "raw_data/rankings_endpoint_2016.json")
make_and_save_api_call(rankings_endpoint, {"year": "2017"}, "raw_data/rankings_endpoint_2017.json")
make_and_save_api_call(rankings_endpoint, {"year": "2018"}, "raw_data/rankings_endpoint_2018.json")

Again, I will read these into DataFrames in the same manner as above:

In [ ]:
rankings_2012 = pd.read_json("raw_data/rankings_endpoint_2012.json")
rankings_2013 = pd.read_json("raw_data/rankings_endpoint_2013.json")
rankings_2014 = pd.read_json("raw_data/rankings_endpoint_2014.json")
rankings_2015 = pd.read_json("raw_data/rankings_endpoint_2015.json")
rankings_2016 = pd.read_json("raw_data/rankings_endpoint_2016.json")
rankings_2017 = pd.read_json("raw_data/rankings_endpoint_2017.json")
rankings_2018 = pd.read_json("raw_data/rankings_endpoint_2018.json")

The [AP Top 25 Poll](https://apnews.com/APTop25CollegeFootballPoll) is one of the most common and therefore I will use it as my proxy for success. This poll is released every week of the college football season, ranking the top 25 teams. There are roughly 130 college football programs so this is a good proxy of the elite programs at a current point of the season. 

However, I really want an indicator of if the team is ranked going into a particular game or not, rather than a list of the top 25 teams. Therefore, I will collect only data from this poll and derive a field to indicate if teams were ranked or not for each game.

First, I will go through the rankings API results and create lists for the teams included in the AP Top 25 poll each week as well as the corresponding season year and week:

In [ ]:
datasets = [rankings_2012, rankings_2013, rankings_2014, rankings_2015, rankings_2016, rankings_2017, rankings_2018]

ap_polls = []
years = []
weeks = []

for dataset in datasets:
    for index, row in dataset.iterrows():
        weeks.append(row.week)
        years.append(row.season)
        polls = row.polls
        for i in range(len(polls)):
            # get poll names and only save AP Top 25 results
            if polls[i]["poll"] == "AP Top 25":
                ranked_teams = polls[i]["ranks"]
                schools = []
                for ranking in ranked_teams:
                    schools.append(ranking["school"])
                ap_polls.append(schools)

I will then add these lists of ranked teams, years and weeks to a DataFrame called `ap_ranking_data`:

In [ ]:
ap_ranking_data = pd.DataFrame()
ap_ranking_data["ap_poll"] = ap_polls 
ap_ranking_data["season"] = years
ap_ranking_data["week"] = weeks
ap_ranking_data.head()

I can now loop through the `attendance_data` DataFrame and add an indicator for both the home and away team in each game to denote if they were ranked when the game occured or not. I will do so by checking to see if the team was included in the AP Top 25 teams for the season and week that the game occurred. A 1 will indicate that the team was ranked when the game occurred and a 0 will indicate the team was unranked.

In [ ]:
home_team_ranking_indicator = []
away_team_ranking_indicator = []

for index, row in attendance_data.iterrows():
    week_rankings = ap_ranking_data[(ap_ranking_data["season"] == row.season) & (ap_ranking_data["week"] == row.week)]
    if row.home_team in week_rankings.ap_poll.iloc[0]:
        home_team_ranking_indicator.append(1)
    else:
        home_team_ranking_indicator.append(0)
        
    if row.away_team in week_rankings.ap_poll.iloc[0]:
        away_team_ranking_indicator.append(1)
    else:
        away_team_ranking_indicator.append(0)

Finally, I will add these two ranking indicators to the `attendance_data` dataset:

In [ ]:
attendance_data["home_team_ranked_ind"] = home_team_ranking_indicator
attendance_data["away_team_ranked_ind"] = away_team_ranking_indicator
attendance_data.head()

---------------------------

### Data cleaning

We can then clean up the data for analysis. First we will filter out any attendance percentages > 1.25, meaning the attendance exceeded the stadium capacity by over 125% which seems highly infeasible and is likely either a data recording error or anomalous event we can exclude for the purposes of this analysis. Then we multiply this feature by 100 so the values are genuine percentages and easier to interpret.

In [ ]:
attendance_data = attendance_data[attendance_data["attendance_percentage"] <= 1.25]
attendance_data["attendance_percentage"] = attendance_data["attendance_percentage"] * 100.0

print("This leaves us with %s rows in the dataset" %str(len(attendance_data)))

Finally, I will convert the temperature values into Fahrenheit for interpretable analysis using the [standard conversion factor](https://www.rapidtables.com/convert/temperature/how-celsius-to-fahrenheit.html):

In [ ]:
attendance_data_with_rain_data["max_temp"] = ((attendance_data_with_rain_data["max_temp"] / 10.0) * (9.0/5.0)) + 32
attendance_data_with_rain_data["min_temp"] = ((attendance_data_with_rain_data["min_temp"] / 10.0) * (9.0/5.0)) + 32

The data is now cleaned and ready for my analysis.

-----------------

### Data aggregation

Finally, I will write the resulting DataFrame to a csv saved in the `processed_data` folder. 

In [ ]:
attendance_data.to_csv("processed_data/seasons_2012-2018_attendance_data.csv")

Analysis of this dataset is conducted in the Data Analysis notebook in this same repository.